In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Load Modules
sys.path.append('/content/drive/MyDrive/Modules')

Mounted at /content/drive


# Plot Routines

# Class definitions

The momentum method allows us to solve the gradient descent problem described above. Looking at the optimization trace above we might intuit that averaging gradients over the past would work well. After all, in the  x1  direction this will aggregate well-aligned gradients, thus increasing the distance we cover with every step. Conversely, in the x2  direction where gradients oscillate, an aggregate gradient will reduce step size due to oscillations that cancel each other out. Using  vt  instead of the gradient  gt  yields the following update equations:

vtxt←βvt−1+gt,t−1,←xt−1−ηtvt. 

Note that for  β=0  we recover regular gradient descent. Before delving deeper into the mathematical properties let's have a quick look at how the algorithm behaves in practice.

In [2]:
import torch.nn as nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.elu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = F.elu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.elu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=4):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet9():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

In [3]:
import random
def plot_transformed_images(image_paths, transform, n=3, seed=42):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)


# **Load Datasets**

In [ ]:
import torchvision.transforms as transforms
from torch.autograd import Variable
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

data_transform=transforms.Compose(
   [ transforms.Resize(size=(32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Setup path to data folder
# Dataset Parameters
data_path = Path('/content/drive/MyDrive/Images/') # The classes or folders under here will be used
image_path = data_path / "allTgts(64x64)"
# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")
    
# Setup train and testing paths
train_dir = image_path / "training"
test_dir = image_path / "test"

# Use ImageFolder to create dataset(s)
trainset = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

testset = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform) 

print(f"Train data:\n{trainset}\nTest data:\n{testset}")

# Get class names as a list
classes = trainset.classes
 
# Turn train and test Datasets into DataLoaders
trainloader = DataLoader(dataset=trainset, 
                              batch_size=default_batch, # how many samples per batch?
                              num_workers=2, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

testloader = DataLoader(dataset=testset, 
                             batch_size=default_batch, 
                             num_workers=2, 
                             shuffle=False) # don't usually need to shuffle testing data


/content/drive/MyDrive/Images/allTgts(64x64) directory exists.
Train data:
Dataset ImageFolder
    Number of datapoints: 3199
    Root location: /content/drive/MyDrive/Images/allTgts(64x64)/training
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 804
    Root location: /content/drive/MyDrive/Images/allTgts(64x64)/test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


# **Original L-BFGS code that works** 

In [ ]:
# Original Code
import torch
import torchvision
import torchvision.transforms as transforms

import math
import time
from lbfgsnew import LBFGSNew 

# Does not error but does not finish.  I suppose this probably takes awhile to train.
# https://github.com/nlesc-dirac/pytorch
# (try to) use a GPU for computation?
use_cuda=True
if use_cuda and torch.cuda.is_available():
  mydevice=torch.device('cuda')
  print ('Using CUDA')
else:
  mydevice=torch.device('cpu')
  print ('Using CPUs')

# try replacing relu with elu
torch.manual_seed(69)
default_batch=128 # no. of batches per epoch 50000/default_batch
batches_for_report=10#
epochs = 20

transform=transforms.Compose(
   [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Using CIFAR-10 images at (3,32,32)
trainset=torchvision.datasets.CIFAR10(root='./torchdata', train=True,
    download=True, transform=transform)

trainloader=torch.utils.data.DataLoader(trainset, batch_size=default_batch,
    shuffle=True, num_workers=2)

testset=torchvision.datasets.CIFAR10(root='./torchdata', train=False,
    download=True, transform=transform)

testloader=torch.utils.data.DataLoader(testset, batch_size=default_batch,
    shuffle=False, num_workers=0)


classes=('plane', 'car', 'bird', 'cat', 
  'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


'''ResNet in PyTorch.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
 
From: https://github.com/kuangliu/pytorch-cifar
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.elu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = F.elu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.elu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.elu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet9():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


# enable this to use wide ResNet
wide_resnet=False
if not wide_resnet:
  net=ResNet9().to(mydevice)
  print("Using resnet9")
else:
  # use wide residual net https://arxiv.org/abs/1605.07146
  net=torchvision.models.resnet.wide_resnet50_2().to(mydevice)
  print("Using resnet50")


#####################################################
def verification_error_check(net):
   correct=0
   total=0
   for data in testloader:
     images,labels=data
     outputs=net(Variable(images).to(mydevice))
     _,predicted=torch.max(outputs.data,1)
     correct += (predicted==labels.to(mydevice)).sum()
     total += labels.size(0)

   return 100*correct//total
#####################################################

lambda1=0.000001
lambda2=0.001

# loss function and optimizer
import torch.optim as optim
#from lbfgsnew import LBFGSNew # custom optimizer
criterion=nn.CrossEntropyLoss()
#optimizer=optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer=optim.Adam(net.parameters(), lr=0.001)
optimizer=LBFGSNew(net.parameters(), history_size=7, max_iter=2, line_search_fn=True,batch_mode=True)

load_model=False
# update from a saved model 
if load_model:
  checkpoint=torch.load('./res18.model',map_location=mydevice)
  net.load_state_dict(checkpoint['model_state_dict'])
  net.train() # initialize for training (BN,dropout)

start_time=time.time()
use_lbfgs=True
# train network
for epoch in range(epochs):  # Setting to 20 Epochs
  running_loss=0.0
  for i,data in enumerate(trainloader,0):
    # get the inputs
    inputs,labels=data
    # wrap them in variable
    inputs,labels=Variable(inputs).to(mydevice),Variable(labels).to(mydevice)

    if not use_lbfgs:
     # zero gradients
     optimizer.zero_grad()
     print("Not using LBFGS!")
     # forward+backward optimize
     outputs=net(inputs)
     loss=criterion(outputs,labels)
     loss.backward()
     optimizer.step()
    else:
      print("Using LBFGS!")
      if not wide_resnet:
        layer1=torch.cat([x.view(-1) for x in net.layer1.parameters()])
        layer2=torch.cat([x.view(-1) for x in net.layer2.parameters()])
        layer3=torch.cat([x.view(-1) for x in net.layer3.parameters()])
        layer4=torch.cat([x.view(-1) for x in net.layer4.parameters()])

      def closure():
        if torch.is_grad_enabled():
         optimizer.zero_grad()
         #print("Using zero gradient")
        outputs=net(inputs)
        if not wide_resnet:
          l1_penalty=lambda1*(torch.norm(layer1,1)+torch.norm(layer2,1)+torch.norm(layer3,1)+torch.norm(layer4,1))
          l2_penalty=lambda2*(torch.norm(layer1,2)+torch.norm(layer2,2)+torch.norm(layer3,2)+torch.norm(layer4,2))
          loss=criterion(outputs,labels)+l1_penalty+l2_penalty
          #print("Apply penalty in Loss function")
        else:
          l1_penalty=0
          l2_penalty=0
          loss=criterion(outputs,labels)
        if loss.requires_grad:
          loss.backward()
          #print('loss %f l1 %f l2 %f'%(loss,l1_penalty,l2_penalty))
        return loss
      optimizer.step(closure)
    # only for diagnostics
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    running_loss +=loss.data.item()

    if math.isnan(loss.data.item()):
       print('loss became nan at %d'%i)
       break

    # print statistics
    if i%(batches_for_report) == (batches_for_report-1): # after every 'batches_for_report'
      print('%f: [%d, %5d] loss: %.5f accuracy: %.3f'%
         (time.time()-start_time,epoch+1,i+1,running_loss/batches_for_report,
         verification_error_check(net)))
      running_loss=0.0

print('Finished Training')


# save model (and other extra items)
torch.save({
            'model_state_dict':net.state_dict(),
            'epoch':epoch,
            'optimizer_state_dict':optimizer.state_dict(),
            'running_loss':running_loss,
           },'./res.model')


# whole dataset
correct=0
total=0
for data in trainloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   
print('Accuracy of the network on the %d train images: %d %%'%
    (total,100*correct//total))

correct=0
total=0
for data in testloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   
print('Accuracy of the network on the %d test images: %d %%'%
    (total,100*correct//total))


class_correct=list(0. for i in range(10))
class_total=list(0. for i in range(10))
for data in testloader:
  images,labels=data
  outputs=net(Variable(images).to(mydevice)).cpu()
  _,predicted=torch.max(outputs.data,1)
  c=(predicted==labels).squeeze()
  for i in range(4):
    label=labels[i]
    class_correct[label] += c[i]
    class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2d %%' %
    (classes[i],100*float(class_correct[i])/float(class_total[i])))


Using CUDA


100%|██████████| 170498071/170498071 [00:01<00:00, 103233483.40it/s]


Extracting ./torchdata/cifar-10-python.tar.gz to ./torchdata
Files already downloaded and verified
Using resnet9
11.332979: [1,    10] loss: 2.24368 accuracy: 19.000
17.497870: [1,    20] loss: 1.84222 accuracy: 28.000
23.134989: [1,    30] loss: 1.62061 accuracy: 34.000
28.792189: [1,    40] loss: 1.60208 accuracy: 31.000
34.864778: [1,    50] loss: 1.63901 accuracy: 37.000
40.413222: [1,    60] loss: 1.52440 accuracy: 31.000
47.001148: [1,    70] loss: 1.56478 accuracy: 29.000
52.600640: [1,    80] loss: 1.49929 accuracy: 40.000
58.895788: [1,    90] loss: 1.40106 accuracy: 41.000
64.839242: [1,   100] loss: 1.38861 accuracy: 37.000
70.679908: [1,   110] loss: 1.42125 accuracy: 36.000
76.555209: [1,   120] loss: 1.38899 accuracy: 43.000
82.183238: [1,   130] loss: 1.39449 accuracy: 33.000
88.617739: [1,   140] loss: 1.37601 accuracy: 43.000
94.166793: [1,   150] loss: 1.28860 accuracy: 39.000
100.718268: [1,   160] loss: 1.37014 accuracy: 42.000
106.470933: [1,   170] loss: 1.24627 a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#L-BFGS DRFM v1 
 

*   Accuracy of the network on the 3199 train images: 99 %
*   Accuracy of the network on the 804 test images: 39 %
*   Accuracy of n01-randomTgts : 50 %
*   Accuracy of n02-rangeTgts : 62 %
*   Accuracy of n03-dopTgts : 75 %
*   Accuracy of n04-combinedTgts : 75 %


In [ ]:
import math
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# https://github.com/nlesc-dirac/pytorch
# (try to) use a GPU for computation?
use_cuda=True
if use_cuda and torch.cuda.is_available():
  mydevice=torch.device('cuda')
  print ('Using CUDA')
else:
  mydevice=torch.device('cpu')
  print ('Using CPUs')

# try replacing relu with elu
torch.manual_seed(69)
default_batch=128 # no. of batches per epoch 50000/default_batch
batches_for_report=10 #
epochs = 100

data_transform=transforms.Compose(
   [ transforms.Resize(size=(32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Setup path to data folder
# Dataset Parameters
data_path = Path('/content/drive/MyDrive/Images/') # The classes or folders under here will be used
image_path = data_path / "reducedTgts(64x64)"
# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory")
    
# Setup train and testing paths
train_dir = image_path / "training"
test_dir = image_path / "test"

# Use ImageFolder to create dataset(s)
trainset = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

testset = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform) 

print(f"Train data:\n{trainset}\nTest data:\n{testset}")

# Get class names as a list
classes = trainset.classes
num_classes= len(classes)
 
# Turn train and test Datasets into DataLoaders
trainloader = DataLoader(dataset=trainset, 
                              batch_size=default_batch, # how many samples per batch?
                              num_workers=2, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

testloader = DataLoader(dataset=testset, 
                             batch_size=default_batch, 
                             num_workers=2, 
                             shuffle=False) # don't usually need to shuffle testing data


'''ResNet in PyTorch.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
 
From: https://github.com/kuangliu/pytorch-cifar'''
# enable this to use wide ResNet
wide_resnet=False

if not wide_resnet:
  net=ResNet9().to(mydevice) # Fewest layers using Resnet9
else:
  # use wide residual net https://arxiv.org/abs/1605.07146
  net=torchvision.models.resnet.resnet152().to(mydevice)

#####################################################
def verification_error_check(net):
   correct=0
   total=0
   for data in testloader:
     images,labels=data
     outputs=net(Variable(images).to(mydevice))
     _,predicted=torch.max(outputs.data,1)
     correct += (predicted==labels.to(mydevice)).sum()
     total += labels.size(0)

   return 100*correct//total
#####################################################

lambda1=0.000001
lambda2=0.001

# loss function and optimizer
import torch.optim as optim
from LBFGS import LBFGS # custom optimizer
criterion=nn.CrossEntropyLoss()
#optimizer=optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer=optim.Adam(net.parameters(), lr=0.001)
#optimizer = LBFGS(net.parameters(), history_size=7, max_iter=2, line_search_fn=True,batch_mode=True)
optimizer = LBFGS(net.parameters(), history_size=7)
print(net.parameters)

### load_model=False
# # update from a saved model 
# if load_model:
#   checkpoint=torch.load('./res18.model',map_location=mydevice)
#   net.load_state_dict(checkpoint['model_state_dict'])
#   net.train() # initialize for training (BN,dropout)

start_time=time.time()
use_lbfgs=True
# train network
for epoch in range(epochs ):
  running_loss=0.0
  for i,data in enumerate(trainloader,0):
    # get the inputs
    inputs,labels=data
    # wrap them in variable
    inputs,labels=Variable(inputs).to(mydevice),Variable(labels).to(mydevice)

    if not use_lbfgs:
     # zero gradients
     optimizer.zero_grad()
     # forward+backward optimize
     outputs=net(inputs)
     loss=criterion(outputs,labels)
     loss.backward()
     optimizer.step()
    else:
      if not wide_resnet:
        layer1=torch.cat([x.view(-1) for x in net.layer1.parameters()])
        layer2=torch.cat([x.view(-1) for x in net.layer2.parameters()])
        layer3=torch.cat([x.view(-1) for x in net.layer3.parameters()])
        layer4=torch.cat([x.view(-1) for x in net.layer4.parameters()])

      def closure():
        if torch.is_grad_enabled():
         optimizer.zero_grad()   # Doing this
        outputs=net(inputs)
        if not wide_resnet:  # Doing this
          l1_penalty=lambda1*(torch.norm(layer1,1)+torch.norm(layer2,1)+torch.norm(layer3,1)+torch.norm(layer4,1))
          l2_penalty=lambda2*(torch.norm(layer1,2)+torch.norm(layer2,2)+torch.norm(layer3,2)+torch.norm(layer4,2))
          loss=criterion(outputs,labels)+l1_penalty+l2_penalty
        else:
          l1_penalty=0
          l2_penalty=0
          loss=criterion(outputs,labels)
        if loss.requires_grad:  # Doing this
          loss.backward()
          #print('loss %f l1 %f l2 %f'%(loss,l1_penalty,l2_penalty))
        return loss
      optimizer.step(closure)
    # only for diagnostics
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    running_loss +=loss.data.item()

    if math.isnan(loss.data.item()):
       print('loss became nan at %d'%i)
       break

    # print statistics
    if i%(batches_for_report) == (batches_for_report-1): # after every 'batches_for_report'
      print('%f: [%d, %5d] loss: %.5f accuracy: %.3f'%
         (time.time()-start_time,epoch+1,i+1,running_loss/batches_for_report,
         verification_error_check(net)))
      running_loss=0.0

print('Finished Training')


# save model (and other extra items)
torch.save({
            'model_state_dict':net.state_dict(),
            'epoch':epoch,
            'optimizer_state_dict':optimizer.state_dict(),
            'running_loss':running_loss,
           },'./res.model')


# whole dataset
correct=0
total=0
for data in trainloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   
print('Accuracy of the network on the %d train images: %d %%'%
    (total,100*correct//total))

correct=0
total=0
for data in testloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   
print('Accuracy of the network on the %d test images: %d %%'%
    (total,100*correct//total))


class_correct=list(0. for i in range(num_classes))
class_total=list(0. for i in range(num_classes))
for data in testloader:
  images,labels=data
  outputs=net(Variable(images).to(mydevice)).cpu()
  _,predicted=torch.max(outputs.data,1)
  c=(predicted==labels).squeeze()
  for i in range(4):
    label=labels[i]
    class_correct[label] += c[i]
    class_total[label] += 1

for i in range(num_classes):
  print('Accuracy of %5s : %2d %%' %
    (classes[i],100*float(class_correct[i])/float(class_total[i])))

Using CUDA
/content/drive/MyDrive/Images/reducedTgts(64x64) directory exists.
Train data:
Dataset ImageFolder
    Number of datapoints: 80
    Root location: /content/drive/MyDrive/Images/reducedTgts(64x64)/training
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 44
    Root location: /content/drive/MyDrive/Images/reducedTgts(64x64)/test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, mo

TypeError: ignored

# L-BFGS DRFM v2

In [ ]:
import math
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# https://github.com/nlesc-dirac/pytorch

use_cuda=True
if use_cuda and torch.cuda.is_available():
  mydevice=torch.device('cuda')
  print ('Using CUDA')
else:
  mydevice=torch.device('cpu')
  print ('Using CPUs')

# try replacing relu with elu
torch.manual_seed(69)
default_batch=128 # no. of batches per epoch 50000/default_batch
batches_for_report=10 #
epochs = 10

# enable this to use wide ResNet
wide_resnet=False

if not wide_resnet:
  net=ResNet9().to(mydevice) # Fewest layers using Resnet9
else:
  # use wide residual net https://arxiv.org/abs/1605.07146
  net=torchvision.models.resnet.wide_resnet50_2().to(mydevice)

#####################################################
def verification_error_check(net):
   correct=0
   total=0
   for data in testloader:
     images,labels=data
     outputs=net(Variable(images).to(mydevice))
     _,predicted=torch.max(outputs.data,1)
     correct += (predicted==labels.to(mydevice)).sum()
     total += labels.size(0)

   return 100*correct//total
#####################################################

lambda1=0.000001
lambda2=0.001

# loss function and optimizer
import torch.optim as optim
from lbfgsnew import LBFGSNew # custom optimizer
criterion=nn.CrossEntropyLoss()
#optimizer=optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer=optim.Adam(net.parameters(), lr=0.001)
optimizer = LBFGSNew(net.parameters(), history_size=7, max_iter=2, line_search_fn=True,batch_mode=True)
print(net.parameters)

### load_model=False
# # update from a saved model 
# if load_model:
#   checkpoint=torch.load('./res18.model',map_location=mydevice)
#   net.load_state_dict(checkpoint['model_state_dict'])
#   net.train() # initialize for training (BN,dropout)

start_time=time.time()
use_lbfgs=True
# train network
for epoch in range(epochs ):
  running_loss=0.0
  for i,data in enumerate(trainloader,0):
    # get the inputs
    print("Enumerate: Train loader loop")
    inputs,labels=data
    # wrap them in variable
    inputs,labels=Variable(inputs).to(mydevice),Variable(labels).to(mydevice)

    if not use_lbfgs:
     # zero gradients
     optimizer.zero_grad()
     # forward+backward optimize
     outputs=net(inputs)
     loss=criterion(outputs,labels)
     loss.backward()
     optimizer.step()
    else:
      if not wide_resnet:
        layer1=torch.cat([x.view(-1) for x in net.layer1.parameters()])
        layer2=torch.cat([x.view(-1) for x in net.layer2.parameters()])
        layer3=torch.cat([x.view(-1) for x in net.layer3.parameters()])
        layer4=torch.cat([x.view(-1) for x in net.layer4.parameters()])

      def closure():
        if torch.is_grad_enabled():
         optimizer.zero_grad()   # Doing this
        outputs=net(inputs)
        if not wide_resnet:  # Doing this
          l1_penalty=lambda1*(torch.norm(layer1,1)+torch.norm(layer2,1)+torch.norm(layer3,1)+torch.norm(layer4,1))
          l2_penalty=lambda2*(torch.norm(layer1,2)+torch.norm(layer2,2)+torch.norm(layer3,2)+torch.norm(layer4,2))
          loss=criterion(outputs,labels)+l1_penalty+l2_penalty
        else:
          l1_penalty=0
          l2_penalty=0
          loss=criterion(outputs,labels)
        if loss.requires_grad:  # Doing this
          loss.backward()
          #print('loss %f l1 %f l2 %f'%(loss,l1_penalty,l2_penalty))
        return loss
      optimizer.step(closure)
    # only for diagnostics
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    running_loss +=loss.data.item()

    if math.isnan(loss.data.item()):
       print('loss became nan at %d'%i)
       break

    # print statistics
    if i%(batches_for_report) == (batches_for_report-1): # after every 'batches_for_report'
      print('%f: [%d, %5d] loss: %.5f accuracy: %.3f'%
         (time.time()-start_time,epoch+1,i+1,running_loss/batches_for_report,
         verification_error_check(net)))
      running_loss=0.0

print('Finished Training')


# save model (and other extra items)
torch.save({
            'model_state_dict':net.state_dict(),
            'epoch':epoch,
            'optimizer_state_dict':optimizer.state_dict(),
            'running_loss':running_loss,
           },'./res.model')

# whole dataset
correct=0
total=0
for data in trainloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   print("Train loader loop")
   
print('Accuracy of the network on the %d train images: %d %%'%
    (total,100*correct//total))

correct=0
total=0
for data in testloader:
   images,labels=data
   outputs=net(Variable(images).to(mydevice)).cpu()
   _,predicted=torch.max(outputs.data,1)
   total += labels.size(0)
   correct += (predicted==labels).sum()
   
print('Accuracy of the network on the %d test images: %d %%'%
    (total,100*correct//total))


class_correct=list(0. for i in range(4))
class_total=list(0. for i in range(4))
for data in testloader:
  images,labels=data
  outputs=net(Variable(images).to(mydevice)).cpu()
  _,predicted=torch.max(outputs.data,1)
  c=(predicted==labels).squeeze()
  for i in range(4):
    label=labels[i]
    class_correct[label] += c[i]
    class_total[label] += 1

for i in range(4):
  print('Accuracy of %5s : %2d %%' %
    (classes[i],100*float(class_correct[i])/float(class_total[i])))

Using CUDA
<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1,

ZeroDivisionError: ignored

# **Full batch L-BFGS (simple equation)**
About 60% accuracy

In [ ]:
"""
Full-Overlap L-BFGS Implementation with Stochastic Wolfe Line Search

Demonstrates how to implement full-overlap L-BFGS with stochastic weak Wolfe line
search without Powell damping to train a simple convolutional neural network using the 
LBFGS optimizer. Full-overlap L-BFGS is a stochastic quasi-Newton method that uses 
the same sample as the one used in the stochastic gradient to perform quasi-Newton 
updating, then resamples an entirely independent new sample in the next iteration.

This implementation is CUDA-compatible.

Implemented by: Hao-Jun Michael Shi and Dheevatsa Mudigere
Last edited 10/20/20.

Requirements:
    - Keras (for CIFAR-10 dataset)
    - NumPy
    - PyTorch

Run Command:
    python full_overlap_lbfgs_example.py

Based on stable quasi-Newton updating introduced by Schraudolph, Yu, and Gunter in
"A Stochastic Quasi-Newton Method for Online Convex Optimization" (2007)

"""

import sys
sys.path.append('../../functions/')

import numpy as np
import torch
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.keras.datasets import cifar10 # to load dataset

from utils import compute_stats, get_grad
from LBFGS import LBFGS

# Parameters for L-BFGS training
max_iter = 200
ghost_batch = 128
batch_size = 8192

# Load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

# Define network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Check cuda availability
cuda = torch.cuda.is_available()
    
# Create neural network model
if cuda:
    torch.cuda.manual_seed(2018)
    model = ConvNet().cuda() 
else:
    torch.manual_seed(2018)
    model = ConvNet()
    
# Define helper functions

# Forward pass
if cuda:
    opfun = lambda X: model.forward(torch.from_numpy(X).cuda())
else:
    opfun = lambda X: model.forward(torch.from_numpy(X))

# Forward pass through the network given the input
if cuda:
    predsfun = lambda op: np.argmax(op.cpu().data.numpy(), 1)
else:
    predsfun = lambda op: np.argmax(op.data.numpy(), 1)

# Do the forward pass, then compute the accuracy
accfun = lambda op, y: np.mean(np.equal(predsfun(op), y.squeeze())) * 100

# Define optimizer
optimizer = LBFGS(model.parameters(), lr=1., history_size=10, line_search='Wolfe', debug=True)

# Main training loop
for n_iter in range(max_iter):
    
    # training mode
    model.train()
    
    # sample batch
    random_index = np.random.permutation(range(X_train.shape[0]))
    Sk = random_index[0:batch_size]
    
    # compute initial gradient and objective
    grad, obj = get_grad(optimizer, X_train[Sk], y_train[Sk], opfun)
    
    # two-loop recursion to compute search direction
    p = optimizer.two_loop_recursion(-grad)
            
    # define closure for line search
    def closure():              
        
        optimizer.zero_grad()
        
        if cuda:
            loss_fn = torch.tensor(0, dtype=torch.float).cuda()
        else:
            loss_fn = torch.tensor(0, dtype=torch.float)
        
        for subsmpl in np.array_split(Sk, max(int(batch_size / ghost_batch), 1)):
                        
            ops = opfun(X_train[subsmpl])
            
            if cuda:
                tgts = torch.from_numpy(y_train[subsmpl]).cuda().long().squeeze()
            else:
                tgts = torch.from_numpy(y_train[subsmpl]).long().squeeze()
                
            loss_fn += F.cross_entropy(ops, tgts) * (len(subsmpl) / batch_size)
                        
        return loss_fn
    
    # perform line search step
    options = {'closure': closure, 'current_loss': obj}
    obj, grad, lr, _, _, _, _, _ = optimizer.step(p, grad, options=options)
        
    # curvature update
    optimizer.curvature_update(grad)
    
    # compute statistics
    model.eval()
    train_loss, test_loss, test_acc = compute_stats(X_train, y_train, X_test, y_test, opfun, accfun,
                                                    ghost_batch=128)
            
    # print data
    print('Iter:', n_iter + 1, 'lr:', lr, 'Training Loss:', train_loss, 'Test Loss:', test_loss,
          'Test Accuracy:', test_acc)


170498071/170498071 [==============================] - 13s 0us/step


/content/drive/MyDrive/Modules/LBFGS.py:257: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


Iter: 1 lr: 1.0 Training Loss: 2.3005604695272446 Test Loss: 2.300521916341781 Test Accuracy: 10.889999999999999
Iter: 2 lr: 1.0 Training Loss: 2.291775402274132 Test Loss: 2.2916575891733175 Test Accuracy: 10.28
Iter: 3 lr: 1.0 Training Loss: 2.2845198953819277 Test Loss: 2.2841987432479867 Test Accuracy: 11.980000000000002
Iter: 4 lr: 1.0 Training Loss: 2.269330383758546 Test Loss: 2.2691428013086323 Test Accuracy: 12.540000000000003
Iter: 5 lr: 1.0 Training Loss: 2.2602930858325956 Test Loss: 2.260551500034332 Test Accuracy: 15.54
Iter: 6 lr: 1.0 Training Loss: 2.2395996826696396 Test Loss: 2.2399813642263404 Test Accuracy: 14.600000000000005
Iter: 7 lr: 0.0956545493549986 Training Loss: 2.224844878101348 Test Loss: 2.2254613263368612 Test Accuracy: 14.360000000000008
Iter: 8 lr: 1.0 Training Loss: 2.1973313039636615 Test Loss: 2.198421651649474 Test Accuracy: 17.370000000000005
Iter: 9 lr: 0.06638844340475818 Training Loss: 2.167039254875183 Test Loss: 2.1692809090137484 Test Accur